
<h1 style="text-align: center;">CIÊNCIA DE DADOS</h1>
<h1 style="text-align: center;">Roteiro de Atividade Prática - PROFESSOR</h1>
<br>
<br>

Nome: ______________________________________________________________________________________      

Turma: ______________


**Componente:** Inteligência Artificial
<br>
**Unidade Curricular:** IA Generativa
<br>
**Tema da Semana:** Implantação de LLMs
<br>


# AULA 2

Passo 1: Configuração Inicial

Atualizar bibliotecas necessárias

In [1]:
!pip install -U langchain langchain-huggingface transformers faiss-cpu

Defaulting to user installation because normal site-packages is not writeable


Importar as bibliotecas

In [2]:
import os
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline


C:\ProgramData\anaconda3\envs\SEDUC_SP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vamos criar um banco de documentos que será usado pelo chatbot para buscar informações antes de responder. 

**Professor**: Reforce com os alunos que estamos trabalhando com exemplos em inglês e ele pode utilizar o google tradutor para auxiliar. Estes são exemplos para relacionar a teoria com a prática.

Criando um arquivo com informações para consulta. **Professor** peça aos alunos para testarem diferentes textos, adicionar outros, incrementar, remover partes. Use a criatividade para personalisar e ver os resultados.

In [3]:
if not os.path.exists("dados"):
    os.makedirs("dados")

with open("dados/documento.txt", "w", encoding="utf-8") as f:
    f.write("XYZ Company Privacy Policy\n\n"
            "1. Information Collection: XYZ Company collects personal information to improve its services.\n"
            "2. Use of Information: Data is used for customer support and recommendations.\n"
            "3. Data Sharing: We do not share information without consent.\n"
            "4. Security: We apply encryption to protect data.\n"
            "5. Updates: This policy may be changed and will be available on the website.")


Agora, vamos configurar um LLM para consultar nossa base de conhecimento antes de gerar respostas.

In [4]:
# Criando os embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Carregando os documentos e dividindo em pedaços menores
loader = TextLoader("dados/documento.txt")
documentos = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documentos_divididos = text_splitter.split_documents(documentos)

# Criando banco de vetores FAISS
vectorstore = FAISS.from_documents(documentos_divididos, embeddings)
retriever = vectorstore.as_retriever()

# Criando um modelo LLM leve
llm_pipeline = pipeline("text-generation", model="gpt2", max_new_tokens=100, temperature=0.7, pad_token_id=50256)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Criando a cadeia de recuperação e resposta
qa = RetrievalQA.from_llm(llm=llm, retriever=retriever)

# Testando o chatbot com RAG
pergunta = "What is the company's privacy policy?"
resposta = qa.invoke({"query": pergunta})

print(f"\nPergunta/Query:\n", resposta['query'])
print(f"\nResultado:\n", resposta['result'])



C:\Users\Daniel Viana\AppData\Local\Temp\ipykernel_24800\1803211535.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
Device set to use cpu



Pergunta/Query:
 What is the company's privacy policy?

Resultado:
 Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
XYZ Company Privacy Policy

1. Information Collection: XYZ Company collects personal information to improve its services.
2. Use of Information: Data is used for customer support and recommendations.
3. Data Sharing: We do not share information without consent.
4. Security: We apply encryption to protect data.
5. Updates: This policy may be changed and will be available on the website.

Question: What is the company's privacy policy?
Helpful Answer:

XYZ Company requires that you provide the following information and that you understand that, under no circumstances will XYZ Company share any information with any third party.

The following information is used in the following ways:

* For the purpose of this policy XYZ Company uses data you provide

# AULA 3

In [5]:
!pip install -U langchain langchain-community langchain-huggingface faiss-cpu transformers


Defaulting to user installation because normal site-packages is not writeable


In [6]:
import os
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

# Criar embeddings com modelo leve do Hugging Face
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Criar uma nova base de conhecimento com informações sobre Termos de Uso e Segurança
if not os.path.exists("dados"):
    os.makedirs("dados")

with open("dados/documento2.txt", "w", encoding="utf-8") as f:
    f.write("XYZ Platform Terms of Use and Security\n\n"
        "1. **Use of the Platform:** The user agrees to use the services of the XYZ Platform only for " "lawful purposes and in accordance with the established terms. Any attempt at fraudulent use will result in " "immediate blocking of the account.\n\n"
        "2. **Data Privacy:** All information collected is stored securely and is never " "shared with third parties without the user's consent. The data is protected by encryption "
        "and follows the standards of the General Data Protection Law (LGPD).\n\n"
        "3. **Information Security:** The XYZ Platform implements two-factor authentication (2FA) to "
        "protect access to accounts. We recommend that users enable this functionality to increase "
        "security.\n\n"
        "4. **User Rights:** Users have the right to request the deletion of their data at any "
        "time. To do so, simply contact us by email at suporte@xyz.com.\n\n"
        "5. **Updates to the Terms:** The XYZ Platform may update its terms of use periodically. "
        "Users will be notified of relevant changes via email or directly on the platform.\n")

print(" Arquivo 'documento2.txt' criado com sucesso!")

# Carregar os documentos e dividir em pedaços menores para melhor recuperação
loader = TextLoader("dados/documento2.txt")
documentos = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documentos_divididos = text_splitter.split_documents(documentos)

# Criar banco de vetores FAISS para recuperação eficiente
vectorstore = FAISS.from_documents(documentos_divididos, embeddings)
retriever = vectorstore.as_retriever()

# Criar um modelo LLM leve para rodar no Google Colab sem consumir muita memória
llm_pipeline = pipeline("text-generation", model="gpt2", max_new_tokens=100, temperature=0.7, pad_token_id=50256)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Criar a cadeia de recuperação e resposta usando LangChain e FAISS
qa = RetrievalQA.from_llm(llm=llm, retriever=retriever)

# Teste do sistema com perguntas baseadas na nova base de conhecimento
print("\n Testando o chatbot com recuperação de conhecimento auxiliada (RAG)...\n")

perguntas = [
    "What are the rights of users on the XYZ Platform?",
    "Does the XYZ Platform share my data with third parties?",
    "How can I protect my account?",
    "What happens if I violate the terms of use?"
]

for pergunta in perguntas:
    resposta = qa.invoke({"query": pergunta})
    print(f"\n Pergunta: {pergunta}")
    print(f"\nPergunta/Query:\n", resposta['query'])
    print(f"\nResultado:\n", resposta['result'])


 Arquivo 'documento2.txt' criado com sucesso!


Device set to use cpu



 Testando o chatbot com recuperação de conhecimento auxiliada (RAG)...


 Pergunta: What are the rights of users on the XYZ Platform?

Pergunta/Query:
 What are the rights of users on the XYZ Platform?

Resultado:
 Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
XYZ Platform Terms of Use and Security

1. **Use of the Platform:** The user agrees to use the services of the XYZ Platform only for lawful purposes and in accordance with the established terms. Any attempt at fraudulent use will result in immediate blocking of the account.

Context:
2. **Data Privacy:** All information collected is stored securely and is never shared with third parties without the user's consent. The data is protected by encryption and follows the standards of the General Data Protection Law (LGPD).

3. **Information Security:** The XYZ Platform implements two-factor authentication (2FA